In [4]:
"""Reference TV reconstruction for ellipse data."""

import numpy as np
import odl
from odl.contrib import fom


In [5]:


# Create ODL data structures
size = 128
space = odl.uniform_discr([-64, -64], [64, 64], [size, size],
                          dtype='float32')

# Creat parallel beam geometry
geometry = odl.tomo.parallel_beam_geometry(space, num_angles=30)

# Create ray transform operator
operator = odl.tomo.RayTransform(space, geometry)

# Create pseudoinverse
pseudoinverse = odl.tomo.fbp_op(operator)



In [6]:

# --- Generate artificial data --- #


# Create phantom
phantom = odl.phantom.shepp_logan(space, modified=True)

# Create sinogram of forward projected phantom with noise
data = operator(phantom)
data += odl.phantom.white_noise(operator.range) * np.mean(np.abs(data)) * 0.05



In [16]:

# --- Set up the inverse problem --- #


# Initialize gradient operator
gradient = odl.Gradient(space)

# Take a sample element from the domain of operator and gradient
# Adjust sample input to match the domain's shape
sample = operator.domain.element(np.ones(operator.domain.shape))


# Apply operator and gradient to check output types
operator_output = operator(sample)
gradient_output = gradient(sample)

print("Operator output type:", type(operator_output))
print("Gradient output type:", type(gradient_output))
print("Operator output :", (operator_output))
print("Gradient output :", (gradient_output))


Operator output type: <class 'odl.discr.lp_discr.DiscreteLpElement'>
Gradient output type: <class 'odl.space.pspace.ProductSpaceElement'>
Operator output : [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
 [ 0.,  0.,  0., ...,  0.,  0.,  0.],
 [ 0.,  0.,  0., ...,  0.,  0.,  0.],
 ..., 
 [ 0.,  0.,  0., ...,  0.,  0.,  0.],
 [ 0.,  0.,  0., ...,  0.,  0.,  0.],
 [ 0.,  0.,  0., ...,  0.,  0.,  0.]]
Gradient output : ProductSpace(uniform_discr([-64., -64.], [ 64.,  64.], (128, 128), dtype='float32'), 2).element([
    
        [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         ..., 
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [-1., -1., -1., ..., -1., -1., -1.]],
    
        [[ 0.,  0.,  0., ...,  0.,  0., -1.],
         [ 0.,  0.,  0., ...,  0.,  0., -1.],
         [ 0.,  0.,  0., ...,  0.,  0., -1.],
         ..., 
         [ 0.,  0.,  0., ...,  0.,  0.

In [17]:

# --- Set up the inverse problem --- #


# Initialize gradient operator
gradient = odl.Gradient(space)

# Column vector of two operators
op = odl.BroadcastOperator(operator, gradient)

# Do not use the g functional, set it to zero.
f = odl.solvers.ZeroFunctional(op.domain)

# Create functionals for the dual variable

# l2-squared data matching
l2_norm = odl.solvers.L2NormSquared(operator.range).translated(data)

# Isotropic TV-regularization i.e. the l1-norm
l1_norm = 0.3 * odl.solvers.L1Norm(gradient.range)

# Combine functionals, order must correspond to the operator K
g = odl.solvers.SeparableSum(l2_norm, l1_norm)



ValueError: object dtype is not supported by sparse matrices

In [ ]:

# --- Select solver parameters and solve using Chambolle-Pock --- #

# Optionally pass callback to the solver to display intermediate results
callback = (odl.solvers.CallbackPrint(lambda x: fom.psnr(x, phantom)) &
            odl.solvers.CallbackShow(clim=[0.1, 0.4]))

# Choose a starting point
x = pseudoinverse(data)

# Run the algorithm
odl.solvers.pdhg(
    x, f, g, op, niter=1000, gamma=0.3,
    callback=None)

print('psnr = {}'.format(fom.psnr(x, phantom)))

# Display images
x.show('Shepp-Logan TV windowed', clim=[0.1, 0.4])
